### Try 1

In [2]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Adjust based on your spatial domain
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Adjust based on your temporal domain

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model, x_train[None], t_train[None], x_train[None], t_train[None])

W0000 00:00:1731990115.787535    7678 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (1, 1225)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 1225, 1), dtype=float32)
  • training=None
  • mask=None

### Try 2:

In [4]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))

    # Create meshgrid for training data (combine spatial and temporal)
    X_train = np.array(np.meshgrid(x_train.flatten(), t_train.flatten())).T.reshape(-1, 2)  # Shape (n_samples, 2)

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model,
               X_train,
               np.tile(t_train.flatten(), (clean_data.shape[1], 1)).T.flatten()[:, None], 
               X_train,
               np.tile(t_train.flatten(), (noisy_data.shape[1], 1)).T.flatten()[:, None])

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 3

In [6]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))

    # Create meshgrid for training data (combine spatial and temporal)
    X_train = np.array(np.meshgrid(x_train.flatten(), t_train.flatten())).T.reshape(-1, 2)  # Shape (n_samples, 2)

    # Prepare corresponding temporal inputs for clean and noisy data
    T_clean = np.tile(t_train.flatten(), (clean_data.shape[1], 1)).T.flatten()[:, None]
    T_noisy = np.tile(t_train.flatten(), (noisy_data.shape[1], 1)).T.flatten()[:, None]

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model,
               X_train,
               T_clean,
               X_train,
               T_noisy)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 4

In [8]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))

    # Create meshgrid for training data (combine spatial and temporal)
    X_train = np.array(np.meshgrid(x_train.flatten(), t_train.flatten())).T.reshape(-1, 2)  # Shape (n_samples, 2)

    # Prepare corresponding temporal inputs for clean and noisy data
    T_clean = np.tile(t_train.flatten(), (clean_data.shape[1], 1)).T.flatten()[:, None]
    T_noisy = np.tile(t_train.flatten(), (noisy_data.shape[1], 1)).T.flatten()[:, None]

    # Ensure that X_train has the correct shape for both clean and noisy datasets.
    assert X_train.shape[1] == 2  # Check that X_train has two columns

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model,
               X_train,
               T_clean,
               X_train,
               T_noisy)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_9" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 5:

In [10]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))

    # Create meshgrid for training data (combine spatial and temporal)
    X_train = np.array(np.meshgrid(x_train.flatten(), t_train.flatten())).T.reshape(-1, 2)  # Shape (n_samples, 2)

    # Prepare corresponding temporal inputs for clean and noisy data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape (205824,)
    T_noisy = np.tile(t_train.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape (205824,)
    
    # Ensure that X_train has the correct shape for both clean and noisy datasets.
    assert X_train.shape[1] == 2  # Check that X_train has two columns

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model,
               X_train,
               T_clean,
               X_train,
               T_noisy)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_12" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 6:

In [12]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train_clean, t_train_clean, x_train_noisy, t_train_noisy):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Training on clean data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_clean = physics_loss(model, x_train_clean, t_train_clean)
        
        grads = tape.gradient(loss_clean, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Clean Loss: {loss_clean.numpy()}')

    # Training on noisy data
    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss_noisy = physics_loss(model, x_train_noisy, t_train_noisy)
        
        grads = tape.gradient(loss_noisy, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Noisy Loss: {loss_noisy.numpy()}')

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))

    # Create meshgrid for training data (combine spatial and temporal)
    X_train = np.array(np.meshgrid(x_train.flatten(), t_train.flatten())).T.reshape(-1, 2)  # Shape (n_samples, 2)

    # Prepare corresponding temporal inputs for clean and noisy data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    T_noisy = np.tile(t_train.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness
    print("X_train shape:", X_train.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)      # Should be (205824, 1)
    print("T_noisy shape:", T_noisy.shape)      # Should be (205824, 1)

    # Ensure that X_train has the correct shape for both clean and noisy datasets.
    assert X_train.shape[1] == 2  # Check that X_train has two columns

    # Create PINN model
    pinn_model = create_pinn_model(input_shape=(2,))
    
    # Train PINN on clean data first then on noisy data
    train_pinn(pinn_model,
               X_train,
               T_clean,
               X_train,
               T_noisy)

X_train shape: (205824, 2)
T_clean shape: (205824, 1)
T_noisy shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_15" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 7:

In [15]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

    # Prepare temporal inputs for noisy data
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201, 1))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
               X_train_clean,
               np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None])   # Flattened temporal input

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                     X_train_clean,
                     np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None])

    
    print("Training on noisy data...")
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
               X_train_noisy,
               np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None])   # Flattened temporal input

   # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                    X_train_noisy,
                    np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None])

Training on clean data...


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_18" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 8:

In [19]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201, 1))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

    # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
               X_train_clean,
               T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                     X_train_clean,
                     T_clean)

    
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

   # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
   
   # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
   
   # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
              X_train_noisy,
              T_noisy)

   # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                    X_train_noisy,
                    T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_21" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 9

In [20]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_24" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 10:

In [23]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_27" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 11

In [24]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024, 1))
    
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_30" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 12

In [1]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024,))
    
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

    # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

2024-11-19 10:20:26.587317: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 10:20:26.596910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731991826.608238    8612 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731991826.611354    8612 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 10:20:26.623154: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


W0000 00:00:1731991828.218981    8612 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 13

In [2]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024,))
    
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_3" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 14

In [5]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))
    
    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)
        
        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)
        
    u_t = tape.gradient(u_pred, t_train)
    
    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)
    
    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))
    
    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")
    
    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()
    
    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  # Spatial domain (shape (1024,))
    
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  # Temporal domain (shape (201,))
    
    # Create meshgrid for training data (combine spatial and temporal)
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 15

In [6]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_9" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### Try 16

In [7]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_12" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 17

In [8]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Check if shapes are correct before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_15" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 18

In [9]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_18" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 19

In [10]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

    # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_21" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 21

In [11]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    # Load data from HDF5 file
    initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

    # Prepare training data (spatial and temporal coordinates)
    
    # Spatial domain (shape (1024,))
    x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
    # Temporal domain for clean data (shape (201,))
    t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
    # Create meshgrid for training data (combine spatial and temporal)
    
    X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
    T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for clean data
    print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
    print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
    
    # Create PINN model for clean data
    pinn_model_clean = create_pinn_model(input_shape=(2,))
    
    print("Training on clean data...")
    
    # Train PINN on clean data
    train_pinn(pinn_model_clean,
                X_train_clean,
                T_clean)

    # Visualize output for clean data
    visualize_output(pinn_model_clean,
                        X_train_clean,
                        T_clean)

        
    print("Training on noisy data...")
    
    # Prepare training data for noisy dataset
    t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
    
    X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

    # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
    T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
    
    # Debugging: Print shapes to verify correctness for noisy data
    print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
    print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
    
    # Create PINN model for noisy data
    pinn_model_noisy = create_pinn_model(input_shape=(2,))
    
    # Train PINN on noisy data
    train_pinn(pinn_model_noisy,
                X_train_noisy,
                T_noisy)

    # Visualize output for noisy data
    visualize_output(pinn_model_noisy,
                        X_train_noisy,
                        T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_24" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 22

In [12]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    
   # Load data from HDF5 file
   initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

   # Prepare training data (spatial and temporal coordinates)
    
   # Spatial domain (shape (1024,))
   x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
   # Temporal domain for clean data (shape (201,))
   t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
   # Create meshgrid for training data (combine spatial and temporal)
    
   X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
   T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for clean data
   print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
   print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
   
   # Create PINN model for clean data
   pinn_model_clean = create_pinn_model(input_shape=(2,))
   
   print("Training on clean data...")
   
   # Train PINN on clean data
   train_pinn(pinn_model_clean,
               X_train_clean,
               T_clean)

   # Visualize output for clean data
   visualize_output(pinn_model_clean,
                    X_train_clean,
                    T_clean)

    
   print("Training on noisy data...")
   
   # Prepare training data for noisy dataset
   t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
   
   X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

   # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
   T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for noisy data
   print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
   print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
   
   # Create PINN model for noisy data
   pinn_model_noisy = create_pinn_model(input_shape=(2,))
   
   # Train PINN on noisy data
   train_pinn(pinn_model_noisy,
              X_train_noisy,
              T_noisy)

   # Visualize output for noisy data
   visualize_output(pinn_model_noisy,
                    X_train_noisy,
                    T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_27" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 23

In [13]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    
   # Load data from HDF5 file
   initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

   # Prepare training data (spatial and temporal coordinates)
    
   # Spatial domain (shape (1024,))
   x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
   # Temporal domain for clean data (shape (201,))
   t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
   # Create meshgrid for training data (combine spatial and temporal)
    
   X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
   T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for clean data
   print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
   print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
   
   # Create PINN model for clean data
   pinn_model_clean = create_pinn_model(input_shape=(2,))
   
   print("Training on clean data...")
   
   # Train PINN on clean data
   train_pinn(pinn_model_clean,
               X_train_clean,
               T_clean)

   # Visualize output for clean data
   visualize_output(pinn_model_clean,
                    X_train_clean,
                    T_clean)

        
   print("Training on noisy data...")
   
   # Prepare training data for noisy dataset
   t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
   
   X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

   # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
   T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for noisy data
   print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
   print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
   
   # Create PINN model for noisy data
   pinn_model_noisy = create_pinn_model(input_shape=(2,))
   
   # Train PINN on noisy data
   train_pinn(pinn_model_noisy,
               X_train_noisy,
               T_noisy)

   # Visualize output for noisy data
   visualize_output(pinn_model_noisy,
                    X_train_noisy,
                    T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_30" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None

### 24

In [14]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load data from HDF5 file
def load_data(file_name):
    with h5py.File(file_name, 'r') as f:
        initial_condition = f['0/initial_condition'][()]
        boundary_condition = f['0/boundary_condition'][()]
        clean_data = f['0/clean'][()]
        noisy_data = f['0/noisy'][()]
        du = f['0/du'][()]
        epsilon = f['0/epsilon'][()]
        u0 = f['0/u0'][()]
        noise_level = f['0/noise_level'][()]
        
    return initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level

# Define PINN model
def create_pinn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output layer for solution
    return model

# Physics-informed loss function
def physics_loss(model, x_train, t_train):
    print(f"Physics loss input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Ensure inputs are correctly shaped before concatenation
    if x_train.shape[1] != 2 or t_train.shape[1] != 1:
        raise ValueError(f"Unexpected input shapes: x_train: {x_train.shape}, t_train: {t_train.shape}")

    # Debugging: Check shapes before concatenation
    print(f"Before concatenation - x_train shape: {x_train.shape}, t_train shape: {t_train.shape}")

    # Calculate predictions
    u_pred = model(tf.concat([x_train, t_train], axis=1))

    # Compute gradients w.r.t. x and t
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x_train)
        tape.watch(t_train)

        u_pred = model(tf.concat([x_train, t_train], axis=1))
        u_x = tape.gradient(u_pred, x_train)

    u_t = tape.gradient(u_pred, t_train)

    # Burgers' equation: u_t + u * u_x - nu * u_xx = 0
    nu = epsilon  # Viscosity coefficient
    u_xx = tape.gradient(u_x, x_train)

    # Physics loss based on Burgers' equation
    loss = tf.reduce_mean(tf.square(u_t + u_pred * u_x - nu * u_xx))

    del tape  # Clean up
    return loss

# Training function
def train_pinn(model, x_train, t_train):
    print(f"Training input shapes - x_train: {x_train.shape}, t_train: {t_train.shape}")

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(10000):  # Adjust epochs as needed
        with tf.GradientTape() as tape:
            loss = physics_loss(model, x_train, t_train)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 1000 == 0:  # Print every 1000 epochs
            print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Visualization function
def visualize_output(model, x_data, t_data):
    print(f"Visualizing output - x_data shape: {x_data.shape}, t_data shape: {t_data.shape}")

    u_pred = model(tf.concat([x_data, t_data], axis=1)).numpy()

    plt.figure(figsize=(10, 6))
    plt.imshow(u_pred.reshape((t_data.shape[0], x_data.shape[0])), aspect='auto', extent=[0, 1, 0, 1])
    plt.colorbar(label='u(x,t)')
    plt.title('Predicted Solution of Burgers\' Equation')
    plt.xlabel('Spatial Coordinate (x)')
    plt.ylabel('Temporal Coordinate (t)')
    plt.show()

# Main execution flow
if __name__ == "__main__":
    
   # Load data from HDF5 file
   initial_condition, boundary_condition, clean_data, noisy_data, du, epsilon, u0, noise_level = load_data('/home/pes1ug22am100/Documents/Research and Experimentation/KINN Final/PINNdropSilence/simulation_data.h5')

   # Prepare training data (spatial and temporal coordinates)
    
   # Spatial domain (shape (1024,))
   x_train = np.linspace(0, 1, clean_data.shape[1])[:, None]  
    
   # Temporal domain for clean data (shape (201,))
   t_train_clean = np.linspace(0, 1, clean_data.shape[0])[:, None]  
    
   # Create meshgrid for training data (combine spatial and temporal)
    
   X_train_clean = np.array(np.meshgrid(x_train.flatten(), t_train_clean.flatten())).T.reshape(-1, 2)  # Shape (n_samples_clean, 2)

   # Prepare corresponding temporal inputs for clean data (ensure they are of shape (n_samples,))
   T_clean = np.tile(t_train_clean.flatten(), (clean_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for clean data
   print("X_train_clean shape:", X_train_clean.shape)      # Should be (205824, 2)
   print("T_clean shape:", T_clean.shape)                  # Should be (205824,)
   
   # Create PINN model for clean data
   pinn_model_clean = create_pinn_model(input_shape=(2,))
   
   print("Training on clean data...")
   
   # Train PINN on clean data
   train_pinn(pinn_model_clean,
               X_train_clean,
               T_clean)

   # Visualize output for clean data
   visualize_output(pinn_model_clean,
                    X_train_clean,
                    T_clean)

        
   print("Training on noisy data...")
   
   # Prepare training data for noisy dataset
   t_train_noisy = np.linspace(0, 1, noisy_data.shape[0])[:, None]  # Temporal domain for noisy data (shape (201,))
   
   X_train_noisy = np.array(np.meshgrid(x_train.flatten(), t_train_noisy.flatten())).T.reshape(-1, 2)  # Shape (n_samples_noisy, 2)

   # Prepare corresponding temporal inputs for noisy data (ensure they are of shape (n_samples,))
   T_noisy = np.tile(t_train_noisy.flatten(), (noisy_data.shape[1],)).flatten()[:, None]   # Shape should be (205824,)
   
   # Debugging: Print shapes to verify correctness for noisy data
   print("X_train_noisy shape:", X_train_noisy.shape)      # Should be (205824, 2)
   print("T_noisy shape:", T_noisy.shape)                  # Should be (205824,)
   
   # Create PINN model for noisy data
   pinn_model_noisy = create_pinn_model(input_shape=(2,))
   
   # Train PINN on noisy data
   train_pinn(pinn_model_noisy,
               X_train_noisy,
               T_noisy)

   # Visualize output for noisy data
   visualize_output(pinn_model_noisy,
                    X_train_noisy,
                    T_noisy)

X_train_clean shape: (205824, 2)
T_clean shape: (205824, 1)
Training on clean data...
Training input shapes - x_train: (205824, 2), t_train: (205824, 1)
Physics loss input shapes - x_train: (205824, 2), t_train: (205824, 1)
Before concatenation - x_train shape: (205824, 2), t_train shape: (205824, 1)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_33" is incompatible with the layer: expected axis -1 of input shape to have value 2, but received input with shape (205824, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(205824, 3), dtype=float32)
  • training=None
  • mask=None